In [1]:
!git clone https://github.com/KL0224/RetrievalPerson -b new_branch

Cloning into 'RetrievalPerson'...
remote: Enumerating objects: 174, done.
remote: Counting objects: 100% (174/174), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 174 (delta 68), reused 154 (delta 48), pack-reused 0 (from 0)
Receiving objects: 100% (174/174), 18.64 MiB | 41.48 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [2]:
!pip install open_clip_torch==3.2.0
!pip install torchreid==0.2.5
!pip install lap>=0.5.12
!pip install ultralytics==8.3.243
!pip install deep-sort-realtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.7/92.7 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torchreid: filename=torchreid-0.2.5-py3-none-any.whl size=144324 sha256=a8727f5ac4e4e92bd6fca3a402b417b025bc2d0f82575eb6803fcb0e8c1ceaad
  Stored in directory: /root/.cache/pip/wheels/5c/86/ff/80a1b78a90df470cbb12c075bf189ad33f1a41a881cf9e9a09
Successfully built torchreid
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 86.0 MB/s eta 0:00:00


In [3]:
import sys
sys.path.append("/kaggle/working/RetrievalPerson/deep_person_reid")

In [4]:
%cd /kaggle/working/RetrievalPerson

/kaggle/working/RetrievalPerson


In [5]:
!rm -rf deep_person_reid

In [6]:
!git clone https://github.com/baotodale06/deep_person_reid

Cloning into 'deep_person_reid'...
remote: Enumerating objects: 5040, done.
remote: Counting objects: 100% (773/773), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 5040 (delta 698), reused 667 (delta 666), pack-reused 4267 (from 2)
Receiving objects: 100% (5040/5040), 1.83 MiB | 19.96 MiB/s, done.
Resolving deltas: 100% (3530/3530), done.


In [7]:
import os

VIDEO_FOLDER = '/kaggle/input/dataset-person/videos'
OUTPUT_FOLDER = '/kaggle/working/outputs'
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

os.makedirs(os.path.join(OUTPUT_FOLDER, 'frames'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_FOLDER, 'annotated_videos'), exist_ok=True)


crops_folder = os.path.join(OUTPUT_FOLDER, 'crops')
os.makedirs(crops_folder, exist_ok=True)

metadata_folder = os.path.join(OUTPUT_FOLDER, 'metadata')
os.makedirs(metadata_folder, exist_ok=True)

features_folder = os.path.join(OUTPUT_FOLDER, 'features')
os.makedirs(features_folder, exist_ok=True)

In [8]:
import numpy as np
import glob
import os
import PIL.Image as Image
import cv2
import torch
import pickle
from collections import defaultdict
from config import *
from tracking.tracklet import TrackletManager
from tracking.detector_tracker import run_tracking
from sampling.sampler import sample_best_per_window
from models.reid import ReIDModel
from models.clip_model import CLIPModel

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


/usr/local/lib/python3.12/dist-packages/torchreid/reid/metrics/rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(
E0000 00:00:1768150256.893309      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768150256.994733      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768150257.828387      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768150257.828439      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768150257.828442      24 computation_placer.cc:177] computation placer already regis

In [9]:
def save_image_webp(img_bgr, path: str, quality: int = 80, resize_factor: float = 0.5):
    if resize_factor != 1.0:
        img_bgr = cv2.resize(img_bgr, (0, 0), fx=resize_factor, fy=resize_factor)
    img_pil = Image.fromarray(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
    os.makedirs(os.path.dirname(path), exist_ok=True)
    img_pil.save(path, format="WEBP", quality=quality)

def save_crop_webp(crop, path):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    cv2.imwrite(
        path,
        crop,
        [cv2.IMWRITE_WEBP_QUALITY, 100]  # 100 = lossless
    )

def safe_delete(path):
    try:
        if path and os.path.exists(path):
            os.remove(path)
    except Exception as e:
        print(f"[WARN] Failed to delete {path}: {e}")


In [10]:
import cv2
import random
import os

def visualize_video_with_ids(
    video_path,
    tracks_per_frame,
    output_path=None
):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        raise IOError(f"Cannot open video: {video_path}")

    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps    = cap.get(cv2.CAP_PROP_FPS) or 30

    writer = None
    if output_path:
        ext = os.path.splitext(output_path)[1].lower()

        fourcc = cv2.VideoWriter_fourcc(*"XVID")
        
        # if ext == ".avi":
        #     fourcc = cv2.VideoWriter_fourcc(*"XVID")
        # elif ext == ".mp4":
        #     fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        # else:
        #     raise ValueError("Unsupported output format. Use .avi or .mp4")

        writer = cv2.VideoWriter(
            output_path,
            fourcc,
            fps,
            (width, height)
        )

    id_colors = {}
    frame_idx = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        for item in tracks_per_frame.get(frame_idx, []):
            track_id = item["id"]
            x1, y1, x2, y2 = map(int, item["bbox"])

            if track_id not in id_colors:
                random.seed(int(track_id))
                id_colors[track_id] = (
                    random.randint(50, 255),
                    random.randint(50, 255),
                    random.randint(50, 255),
                )

            color = id_colors[track_id]

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(
                frame,
                f"ID {track_id}",
                (x1, max(0, y1 - 7)),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                color,
                2
            )

        if writer:
            writer.write(frame)

        frame_idx += 1

    cap.release()
    if writer:
        writer.release()

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [12]:
# tracking
reid_model = ReIDModel(device=device, model_path='./models/osnet_ain_x1_0_dukemtmcreid_256x128_amsgrad_ep90_lr0.0015_coslr_b64_fb10_softmax_labsmth_flip_jitter.pth')
clip_model = CLIPModel(device=device)

seqs = sorted(glob.glob(f'{VIDEO_FOLDER}/seq_*'))

# seq 002
for seq in seqs[-3:-2]: # !!!!!
    seq_name = os.path.basename(seq)
    seq_id = int(seq_name.split('_')[-1])
    print(f'Processing sequence {seq_name}')
    
    os.makedirs(os.path.join(OUTPUT_FOLDER, 'frames', seq_name), exist_ok=True)
    os.makedirs(os.path.join(OUTPUT_FOLDER, 'annotated_videos', seq_name), exist_ok=True)
    cameras = sorted(glob.glob(f'{seq}/camera_*'))
    for video_path in cameras: 
        manager = TrackletManager()
        camera_name = "_".join(os.path.basename(video_path).split('_')[:2])
        cam_id = int(camera_name[-1])
        camera_frame_folder = os.path.join(OUTPUT_FOLDER, 'frames', seq_name, camera_name)
        os.makedirs(camera_frame_folder, exist_ok=True)
        print(f'  Processing camera {cam_id}')
        for frame_id, frame, boxes, ids, confs in run_tracking(video_path, model_name='yolov8x.pt', 
                                                               vid_stride=1, 
                                                               confidence=CONFIDENCE_THRESHOLD,
                                                               device=device):
            if frame_id%100==0:
                print(f'    Processing frame {frame_id}') 
            # detected boxes + (alive but not detected)
            # print(boxes)
            frame_save_path = os.path.join(camera_frame_folder, f'frame_{frame_id:06d}.webp')
            # save_image_webp(frame, frame_save_path)
            for box, tid, conf in zip(boxes, ids, confs):
                # print(f'      Track ID: {tid}, BBox: {box}, Conf: {conf}')
                gid = seq_id*SEQ_ID_OFFSET + cam_id * CAMERA_ID_OFFSET + tid
                x1, y1, x2, y2 = map(int, box)

                # invalid box
                if x2<=x1 and y2<=y1:
                    continue
                
                crop = frame[y1:y2, x1:x2]

                crop_path = os.path.join(
                    OUTPUT_FOLDER,
                    "crops",
                    seq_name,
                    camera_name,
                    f"{gid}_{frame_id:06d}.webp"
                )
                
                save_crop_webp(crop, crop_path)
                t = manager.get(gid, seq_id, cam_id)
                t.add_frame(frame_id, box, conf, crop_path)

        print('Start SAMPLING and EMBEDDING')
        tracklets = manager.all()

        for t in tracklets:
            candidates = sample_best_per_window(t.frames)
            
            candidate_paths = set(f.crop_path for f in candidates)

            # delete crops that are not used for CLIP & ReID embedding
            for f in t.frames:
                if f.crop_path not in candidate_paths:
                    safe_delete(f.crop_path)
                    f.crop_path = None

            imgs = [cv2.imread(f.crop_path) for f in candidates]
        
            reid_feat = reid_model.extract(imgs).mean(axis=0)
            reid_feat = reid_feat / np.linalg.norm(reid_feat)

            t.reid_embeddings.append(reid_feat)

            
            clip_feat = clip_model.encode_batch(imgs).mean(axis=0)
            clip_feat = clip_feat / np.linalg.norm(clip_feat)
            
            t.clip_embeddings.append(clip_feat)          
        
        
        
        
        tracks = defaultdict(list)
        
        print('Start SAVING metadata')
        # save into metadata.txt
        with open(f"{OUTPUT_FOLDER}/metadata/{seq_name}_{camera_name}.txt", "w") as f:
            for t in tracklets:
                for frame in t.frames:
                    f.write(f"{t.sequence_id} {t.camera_id} {frame.frame_id} {t.global_id} {int(frame.bbox[0])} {int(frame.bbox[1])} {int(frame.bbox[2])} {int(frame.bbox[3])}\n")
                    tracks[frame.frame_id].append({
                        "id": t.global_id,
                        "bbox": frame.bbox
                    })
        print('Start SAVING embeddings')            
        # save into features.txt
        features = {}
        output_path_pkl = f"{OUTPUT_FOLDER}/features/{seq_name}_{camera_name}.pkl"
        
        with open(f"{OUTPUT_FOLDER}/features/{seq_name}_{camera_name}.txt", "w") as f:
            for t in tracklets:
                f.write(f"{t.sequence_id} {t.camera_id} {t.global_id} {t.reid_embeddings[0].tolist()} {t.clip_embeddings[0].tolist()}\n")
                key = (t.sequence_id, t.camera_id, t.global_id)
        
                reid_emb = t.reid_embeddings[0]
                clip_emb = t.clip_embeddings[0]
            
                features[key] = [reid_emb, clip_emb]
                
        with open(output_path_pkl, "wb") as f:
            pickle.dump(features, f, protocol=pickle.HIGHEST_PROTOCOL)

        print('Start SAVING video')
        visualize_video_with_ids(video_path, tracks, f'{os.path.join(OUTPUT_FOLDER, 'annotated_videos', seq_name, camera_name)}.avi')

Model: osnet_x1_0
- params: 2,193,616
- flops: 978,878,352
Successfully loaded pretrained weights from "./models/osnet_ain_x1_0_dukemtmcreid_256x128_amsgrad_ep90_lr0.0015_coslr_b64_fb10_softmax_labsmth_flip_jitter.pth"
** The following layers are discarded due to unmatched keys or layer size: ['conv2.0.conv2.0.layers.0.conv1.weight', 'conv2.0.conv2.0.layers.0.conv2.weight', 'conv2.0.conv2.0.layers.0.bn.weight', 'conv2.0.conv2.0.layers.0.bn.bias', 'conv2.0.conv2.0.layers.0.bn.running_mean', 'conv2.0.conv2.0.layers.0.bn.running_var', 'conv2.0.conv2.0.layers.0.bn.num_batches_tracked', 'conv2.0.conv2.1.layers.0.conv1.weight', 'conv2.0.conv2.1.layers.0.conv2.weight', 'conv2.0.conv2.1.layers.0.bn.weight', 'conv2.0.conv2.1.layers.0.bn.bias', 'conv2.0.conv2.1.layers.0.bn.running_mean', 'conv2.0.conv2.1.layers.0.bn.running_var', 'conv2.0.conv2.1.layers.0.bn.num_batches_tracked', 'conv2.0.conv2.1.layers.1.conv1.weight', 'conv2.0.conv2.1.layers.1.conv2.weight', 'conv2.0.conv2.1.layers.1.bn.weight

open_clip_pytorch_model.bin:   0%|          | 0.00/3.94G [00:00<?, ?B/s]

Processing sequence seq_024
  Processing camera 1
Model: osnet_x1_0
- params: 2,193,616
- flops: 978,878,352
Successfully loaded pretrained weights from "./models/osnet_x1_0_market_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
    Processing frame 0
    Processing frame 100
    Processing frame 200
    Processing frame 300
    Processing frame 400
    Processing frame 500
    Processing frame 600
    Processing frame 700
    Processing frame 800
    Processing frame 900
    Processing frame 1000
    Processing frame 1100
    Processing frame 1200
    Processing frame 1300
    Processing frame 1400
    Processing frame 1500
    Processing frame 1600
    Processing frame 1700
    Processing frame 1800
    Processing frame 1900
    Processing frame 2000
    Processing frame 2100
    Processing frame 2200
    Processing frame 2300
    Processing 

In [13]:
!zip -rq /kaggle/working/results.zip /kaggle/working/outputs